# Projet DataMining

## Description du projet
L'objectif de ce projet est de recommander des images de pokémons en fonction des préférences de l'utilisateur. 

## Installation librairies


In [ ]:
!pip install -r requirements.txt

## Installation des bases de données

#### Authentification
_Pour utiliser l' API publique de Kaggle , vous devez d'abord vous authentifier à l'aide d'un jeton d'API. Depuis l'en-tête du site, cliquez sur votre photo de profil d'utilisateur, puis sur "Mon compte" dans le menu déroulant. Cela vous amènera aux paramètres de votre compte sur https://www.kaggle.com/account. Faites défiler jusqu'à la section de la page intitulée API :_

_Pour créer un nouveau jeton, cliquez sur le bouton "Créer un nouveau jeton API". Cela téléchargera un nouveau jeton d'authentification sur votre machine._

_Si vous utilisez l'outil Kaggle CLI, l'outil recherchera ce jeton dans __~/.kaggle/kaggle.json sous Linux__, OSX et d'autres systèmes d'exploitation basés sur UNIX, et dans __C:\Users<Windows-username>.kaggle\kaggle.json sous Windows__. Si le jeton n'est pas là, une erreur sera levée. Par conséquent, une fois que vous avez téléchargé le jeton, vous devez le déplacer de votre dossier Téléchargements vers ce dossier._

_Si vous utilisez directement l'API Kaggle, l'endroit où vous conservez le jeton n'a pas d'importance, tant que vous êtes en mesure de fournir vos informations d'identification au moment de l'exécution._

In [ ]:
import os
import kaggle
import shutil

In [ ]:

# Télécharger le fichier de données
kaggle.api.authenticate()
kaggle.api.dataset_download_files('kvpratama/pokemon-images-dataset', path='.', unzip=True, quiet=False, force=False)
kaggle.api.dataset_download_files('abcsds/pokemon', path='./data_csv', unzip=True, quiet=False, force=False)
kaggle.api.dataset_download_files('avi1023/color-names', path='./data_csv', unzip=True, quiet=False, force=False)

In [ ]:
# supprimer le dossier pokemon dans pokemon_img
shutil.rmtree('./pokemon')

# Creation de la base de données

## I. Création de la base de données de pokémons

In [ ]:
from PIL import Image
import os
import json
from datetime import datetime
import numpy as np

In [ ]:
img_dir = r"./pokemon_jpg/pokemon_jpg" # chemin vers le répertoire contenant les images

# initialise un dictionnaire pour stocker les métadonnées de toutes les images
all_metadata = {}
color_data={}

In [ ]:
# boucle à travers tous les fichiers d'image dans le répertoire
for img_filename in os.listdir(img_dir):
    if img_filename.endswith(".jpg") or img_filename.endswith(".png"):
        # construire le chemin complet vers le fichier d'image
        img_path = os.path.join(img_dir, img_filename)

        # ouvrir le fichier d'image
        with Image.open(img_path) as img:

            # supprimer les images avec un nom qui ne sont pas de ce format: 1.jpg, 2.jpg, 3.jpg, etc.
            if not img_filename.split(".")[0].isdigit():
                continue

            # extraire les métadonnées de l'image
            img_filename= img.filename
            img_format = img.format
            img_size = img.size
            img_orientation = "landscape" if img_size[0] > img_size[1] else "portrait"
            creation_date =  datetime.fromtimestamp(os.path.getctime(img_path)).strftime('%d/%m/%Y')

            # créer un dictionnaire de métadonnées pour cette image
            metadata = {
                #on veut juste le nom de l'image
                "id": int((img_filename.split("\\")[-1]).split(".")[0]),
                "format": img_format,
                "size": img_size,
                "orientation": img_orientation,
                "creation_date": creation_date,
                "tags": ""
            }

            # ajouter les métadonnées de cette image au dictionnaire de toutes les métadonnées
            all_metadata[img_filename.split("\\")[-1]] = metadata

In [ ]:
# Ajouter les métadonnées au fichier JSON
with open('database.json', "w") as f:
    json.dump(all_metadata, f, indent=4)

## II. Création de la base de données de couleurs

In [ ]:
import pandas as pd
import json

In [ ]:
# charger le fichier csv
df = pd.read_csv("./data_csv/color_names.csv", sep=',', header=0)

# sélectionner les colonnes Name et Hex (24 bit)
df_selected = df.loc[:, ["Name", "Hex (24 bit)"]]
df_selected.rename(columns={"Hex (24 bit)": "Hex"}, inplace=True)

# remplacer # par rien
df_selected['Hex'] = df_selected['Hex'].str.replace('#', '')

with open('color_names.json', 'w') as f:
    f.write(df_selected.to_json(orient='records')) 

with open('color_names.json', 'r') as f:
    data = json.load(f)

with open('color_names.json', 'w') as f:
    json.dump(data, f, indent=4)

## III. Ajout des couleurs aux pokémons

In [ ]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
from sklearn.cluster import MiniBatchKMeans

In [ ]:
# boucle à travers tous les fichiers d'image dans le répertoire
color_data = {}
img_dir = r"./pokemon_jpg/pokemon_jpg" 
for img_filename in os.listdir(img_dir): 
    if img_filename.endswith(".jpg") or img_filename.endswith(".png"):
        # Construire le chemin complet vers le fichier d'image
        img_path = os.path.join(img_dir, img_filename)
        # Ouvrir l'image
        with Image.open(img_path) as img:
            
            
            # Supprimer les images avec un nom qui ne sont pas de ce format: 1.jpg, 2.jpg, 3.jpg, etc.
            if not img_filename.split(".")[0].isdigit():
                continue

            pixel_matrix = np.array(img) # Convertir l'image en matrice de pixels

            # Extraire les valeurs R, G, B
            pixel_data = pixel_matrix.reshape((-1, 3))

            # Utiliser MiniBatchKMeans pour trouver le cluster le plus grand
            kmeans = MiniBatchKMeans(n_clusters=4, random_state=0).fit(pixel_data)
            main_color = kmeans.cluster_centers_[np.argmax(np.unique(kmeans.labels_, return_counts=True)[1])]
            hex_value = "{0:02X}{1:02X}{2:02X}".format(int(main_color[0]), int(main_color[1]), int(main_color[2]))


            # créer un dictionnaire de couleur pour cette image
            color = {
                "id": int((img_filename.split("\\")[-1]).split(".")[0]),
                "couleur dominante": main_color.tolist(),
                "nom couleur": hex_value
            }

            # ajouter les couleurs de cette image au dictionnaire de toutes les couleurs
            color_data[(img_filename.split("\\")[-1]).split(".")[0]] = color


In [ ]:
# Ajouter les couleurs au fichier JSON
with open('color_data.json', 'w') as f:
    json.dump(color_data, f, indent=4)

In [ ]:
# compare les couleurs de l'image avec la liste des couleurs du fichier json

with open('color_names.json', 'r') as f:
    color_names = json.load(f)

with open('color_data.json', 'r') as f:
    color_data = json.load(f)

for pokemon in color_data:
    color_hex = color_data[pokemon]['nom couleur']
    for color in color_names:
        color_rgb = (int(color_hex[1:2], 16), int(color_hex[2:4], 16), int(color_hex[4:], 16))
        color_rgb_names = (int(color['Hex'][1:2], 16), int(color['Hex'][2:4], 16), int(color['Hex'][4:], 16))
        # verifie si les trois valeurs sont les memes
        if (color_rgb[0] == color_rgb_names[0] and color_rgb[1] == color_rgb_names[1] and color_rgb[2] == color_rgb_names[2]):
            color_data[pokemon]['nom couleur'] = color['Name']
        # verifie si deux des trois valeurs sont les memes
        elif ((color_rgb[0] == color_rgb_names[0] and color_rgb[1] == color_rgb_names[1])
        or (color_rgb[0] == color_rgb_names[0] and color_rgb[2] == color_rgb_names[2]) 
        or (color_rgb[1] == color_rgb_names[1] and color_rgb[2] == color_rgb_names[2])):
            color_data[pokemon]['nom couleur'] = color['Name']
        # verifie si une des trois valeurs est la meme
        elif (color_rgb[0] == color_rgb_names[0] or color_rgb[1] == color_rgb_names[1] or color_rgb[2] == color_rgb_names[2]):
            color_data[pokemon]['nom couleur'] = color['Name']

with open('color_data.json', 'w') as f:
    json.dump(color_data, f, indent=4)

In [ ]:
# ajoute les couleurs au fichier json principal
with open('database.json', 'r') as f:
    pokemon_data = json.load(f)

with open('color_data.json', 'r') as f:
    color_data = json.load(f)

# si l'id du pokemon est le meme que l'id de la couleur, ajoute la couleur au pokemon
for pokemon in pokemon_data:
    for color in color_data:
        if pokemon_data[pokemon]['id'] == color_data[color]['id']:
            pokemon_data[pokemon]['couleur dominante'] = color_data[color]['nom couleur']

with open('database.json', 'w') as f:
    json.dump(pokemon_data, f, indent=4)


## Ajout de tags

In [ ]:
import pandas as pd
import json
import csv

In [ ]:
# Charger les données de tags.json dans data_d["tags"]
df = pd.read_csv(r"./data_csv/Pokemon.csv", sep=',', header=0)
df_selected = df.loc[:, ['#','Name', 'Type 1', 'Type 2', 'Generation', 'Legendary']]

In [ ]:
# Renommer la colonne # en id
df_selected.rename(columns={'#': 'id'}, inplace=True)

with open('tags.json', 'w') as f:
    f.write(df_selected.to_json(orient='records')) 

with open('tags.json', 'r') as f:
    data_t = json.load(f)

with open('tags.json', 'w') as f:
    json.dump(data_t, f, indent=4)

with open('database.json', 'r') as f:
    data_d = json.load(f)

In [ ]:
# Charger les données de tags.json dans data_d["tags"]
for key in data_d:
    for i in range(len(data_t)):
        if data_d[key]["id"] == data_t[i]["id"]:
            data_d[key]["tags"] = data_t[i]

#Enlever l'id de data_d["tags"]
for key in data_d:
    data_d[key]["tags"].pop("id")

# Enregistrer les données dans database.json
with open('database.json', 'w') as f:
    json.dump(data_d, f, indent=4)

## Creation des utilisateurs

In [ ]:
import json
from random import randint, choice
import os

In [ ]:
# Fonction de filtrage en fonction des préférences utilisateur
def filter_images(images, Type1, Type2, legendary):
    filtered_images = []
    for image in images.values():
        color = image["couleur dominante"]
        type1 = image["tags"]["Type 1"]
        type2 = image["tags"]["Type 2"]
        legendaire = image["tags"]["Legendary"]
        tags= image["tags"]
        if (type1 == Type1 or type2 == Type1) and (type1 == Type2 or type2 == Type2) and not(legendaire^legendary):
            tags["color"]=color
            tags["first_letter"]=tags["Name"][0]
            filtered_images.append(tags)
    return filtered_images


In [ ]:
# récuperation des nom couleurs dans database.json
data_t={}
with open("database.json", "r") as f:
    data=json.load(f)
    for i in data:
        data_t[data[i]["id"]]=data[i]

# récuperation des Types dans database.json
Type_t=[]
for i in data_t:
    if data_t[i]["tags"]["Type 1"] not in Type_t:
        Type_t.append(data_t[i]["tags"]["Type 1"])
    if data_t[i]["tags"]["Type 2"] not in Type_t:
        Type_t.append(data_t[i]["tags"]["Type 2"])

# récuperation des couleurs dans database.json
color_t=[]
for i in data_t:
    if data_t[i]["couleur dominante"] not in color_t:
        color_t.append(data_t[i]["couleur dominante"])

# récuperation des nom des pokemons dans database.json
name_t=[]
for i in data_t:
    if data_t[i]["tags"]["Name"] not in name_t:
        name_t.append(data_t[i]["tags"]["Name"])

legendary_t=[True, False]


In [ ]:
# enregistrement des données dans recommendation.json
with open("recommendation.json", "w") as f:
    json.dump({"Type":Type_t, "Color":color_t, "Name":name_t}, f, indent=4)

In [ ]:
#simulation de l'utilisateur

favorite_t=["Favorite","NoFavorite"]
all_user={}
nb_user=100
i=0

while i<nb_user:
    result=[]
    Type1=choice(Type_t) # choix du premier type
    Type2=choice(Type_t) # choix du deuxième type
    legendary=choice(legendary_t)
    data=filter_images(data_t, Type1, Type2, legendary) # filtrage des images
    if len(data)>0:
        for k in range(len(data)): # choix de favorite ou non favorite
            result.append(favorite_t[randint(0, len(favorite_t)-1)])
        all_user[i]={"data":data, "result":result}
        i+=1
    else:
        continue

    

In [ ]:
pok=[]
like=[]

for i in all_user: # création d'un dictionnaire avec les données et les résultats
    for k in range(len(all_user[i]["data"])):
        pok.append(all_user[i]["data"][k])
        like.append(all_user[i]["result"][k])

user={"data":pok, "result":like}

In [ ]:
# sauvegarde des données utilisateur
with open("user.json", "w") as f:
    json.dump(user, f, indent=4)

## Entrainement du modèle

## A. Decision Tree

In [ ]:
from sklearn import tree
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import graphviz
import pydotplus
from IPython.display import Image, display
import json
import pickle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pickle

In [ ]:
#importer les données de l'utilisateur
with open('user.json', "r") as f:
        user=json.load(f)

In [ ]:
#creation du tree
dtc = tree.DecisionTreeClassifier()

#creation des labels encoder
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
le4 = LabelEncoder()
le5 = LabelEncoder()
le6 = LabelEncoder()
le7 = LabelEncoder()
le8 = LabelEncoder()

### __A.1 Création du modèle__

In [ ]:
# creation des dataframes
data = user["data"]
result = user["result"]
dataframe = pd.DataFrame(data, columns=["Name","Type 1", "Type 2", "Generation","legendary", "color","first_letter"])
resultframe = pd.DataFrame(result, columns=["favorite"])

# encoding des données
dataframe["Name"] = le1.fit_transform(dataframe["Name"])
dataframe["Type 1"] = le2.fit_transform(dataframe["Type 1"])
dataframe["Type 2"] = le3.fit_transform(dataframe["Type 2"])
dataframe["Generation"] = le4.fit_transform(dataframe["Generation"])
dataframe["legendary"] = le5.fit_transform(dataframe["legendary"])
dataframe["color"] = le6.fit_transform(dataframe["color"])
dataframe["first_letter"] = le7.fit_transform(dataframe["first_letter"])
resultframe["favorite"] = le8.fit_transform(resultframe["favorite"])

# creation du model
dtc.fit(dataframe, resultframe)



### __A.2 Visualisation du modèle__

In [ ]:
# Visualisation du modele 
dot_data = tree.export_graphviz(
    dtc,
    out_file=None,
    feature_names=dataframe.columns,
    filled=True,
    rounded=True,
    class_names=le8.inverse_transform(resultframe.favorite.unique()),
    special_characters=True,
)
graph = graphviz.Source(dot_data)

pydot_graph = pydotplus.graph_from_dot_data(dot_data)
img = Image(pydot_graph.create_png())
display(img)

### __A.3 Evaluation du modèle__

In [ ]:
# precision du modele
print("Accuracy:",metrics.accuracy_score(resultframe, dtc.predict(dataframe)))

In [ ]:
# prediction du modele
y_pred = dtc.predict(dataframe)

# creating confusion matrix
cm = confusion_matrix(resultframe, y_pred, normalize='true')
    
# displaying confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['not favorite', 'favorite'])
disp.plot(cmap='Blues')
plt.show()

### __Sauvegarde du modèle__

In [ ]:
pickle.dump(dtc, open("decision_tree.pkl", "wb"))

## B. Neural Network

### __B.1 Creation du modèle__

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model
import json
import matplotlib.pyplot as plt
import pickle
from ann_visualizer.visualize import ann_viz

In [ ]:
# importation des données utilisateur
with open('user.json', 'r') as f:
    user_data = json.load(f)

In [ ]:
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
le4 = LabelEncoder()
le5 = LabelEncoder()
le6 = LabelEncoder()
le7 = LabelEncoder()
le8 = LabelEncoder()

In [ ]:
# creation des dataframes
data = user_data["data"]
result = user_data["result"]
dataframe = pd.DataFrame(data, columns=["Name","Type 1", "Type 2", "Generation","legendary", "color","first_letter"])
resultframe = pd.DataFrame(result, columns=["favorite"])

# encoding des données
dataframe["Name"] = le1.fit_transform(dataframe["Name"])
dataframe["Type 1"] = le2.fit_transform(dataframe["Type 1"])
dataframe["Type 2"] = le3.fit_transform(dataframe["Type 2"])
dataframe["Generation"] = le4.fit_transform(dataframe["Generation"])
dataframe["legendary"] = le5.fit_transform(dataframe["legendary"])
dataframe["color"] = le6.fit_transform(dataframe["color"])
dataframe["first_letter"] = le7.fit_transform(dataframe["first_letter"])
resultframe["favorite"] = le8.fit_transform(resultframe["favorite"])

# Separe les données en données d'entrainement et de test
train_dataset = dataframe.sample(frac=0.8, random_state=0)
test_dataset = dataframe.drop(train_dataset.index)

# Separe les labels en labels d'entrainement et de test
train_labels = resultframe.sample(frac=0.8, random_state=0)
test_labels = resultframe.drop(train_labels.index)

# Definie le modele
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(7,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilation du modele
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# entrainement du modele
history = model.fit(train_dataset, train_labels, validation_data=(test_dataset, test_labels), epochs=100) # epochs= nombre d'itération


### __B.2 Visualisation du modèle__

In [ ]:
# visualisation du modele
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
ann_viz(model, title="Neural Network Model for Favorite Pokemon", view=True)

### __B.3 Evaluation du modèle__

In [ ]:
# Evaluation du modele
test_loss, test_acc = model.evaluate(train_dataset, train_labels)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

In [ ]:
# affichage des courbes d'entrainement et de validation
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Accuracy and Loss')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.show()

### __Sauvegarde du modèle__

In [ ]:
# sauvegarde du modele
pickle.dump(model, open("neural_network.pkl", "wb"))

## C. Random Forest

### __C.1 Creation du modèle__

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import json
from sklearn.datasets import make_classification
from sklearn.tree import plot_tree
import pickle

In [ ]:
# importation des données utilisateur
with open('user.json', 'r') as f:
    user_data = json.load(f)

# creation des label encoder
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
le4 = LabelEncoder()
le5 = LabelEncoder()
le6 = LabelEncoder()
le7 = LabelEncoder()
le8 = LabelEncoder()

In [ ]:
# creation des dataframes
data = user_data["data"]
result = user_data["result"]
dataframe = pd.DataFrame(data, columns=["Name","Type 1", "Type 2", "Generation","legendary", "color","first_letter"])
resultframe = pd.DataFrame(result, columns=["favorite"])

# encoding des données
dataframe["Name"] = le1.fit_transform(dataframe["Name"])
dataframe["Type 1"] = le2.fit_transform(dataframe["Type 1"])
dataframe["Type 2"] = le3.fit_transform(dataframe["Type 2"])
dataframe["Generation"] = le4.fit_transform(dataframe["Generation"])
dataframe["legendary"] = le5.fit_transform(dataframe["legendary"])
dataframe["color"] = le6.fit_transform(dataframe["color"])
dataframe["first_letter"] = le7.fit_transform(dataframe["first_letter"])
resultframe["favorite"] = le8.fit_transform(resultframe["favorite"])

# Separe les données en données d'entrainement et de test
train_data = dataframe.sample(frac=0.8, random_state=0)
test_data = dataframe.drop(train_data.index)

# Separe les labels en labels d'entrainement et de test
train_labels = resultframe.sample(frac=0.8, random_state=0)
test_labels = resultframe.drop(train_labels.index)

# Definie le modele
model = RandomForestClassifier(n_estimators=100, random_state=0)

# entrainement du modele
model.fit(train_data, train_labels)

### __C.2 Visualisation du modèle__

In [ ]:
plot_tree(model.estimators_[0], filled=True, rounded=True)

### __C.3 Evaluation du modèle__

In [ ]:
# Evaluation du modele
y_pred = model.predict(train_data)
print("Accuracy:", accuracy_score(train_labels, y_pred))

In [ ]:
# afficher la matrice de confusion
cm = confusion_matrix(train_labels, y_pred, normalize='true')

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['not favorite', 'favorite'])
disp.plot(cmap='Blues')
plt.show()

### __Sauvegarde du modèle__

In [ ]:
# sauvegarder le modèle
pickle.dump(model, open("random_forest.pkl", "wb"))

## Syteme de recommandation

In [ ]:
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
import pickle
import json
import random
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def recommend_pokemon():
    satisfait = False # Variable de controle de la boucle

    # import data
    with open('data.json', 'r') as f:
        data = json.load(f)

    with open('recommendations.json', 'r') as f:
        rec= json.load(f)

    # importation des modèles
    random_forest = pickle.load(open("random_forest.pkl", "rb"))
    neural_network = pickle.load(open("neural_network.pkl", "rb"))
    decision_tree = pickle.load(open("decision_tree.pkl", "rb"))

    # creation des label encoder
    le1 = LabelEncoder()
    le2 = LabelEncoder()
    le3 = LabelEncoder()
    le4 = LabelEncoder()
    le5 = LabelEncoder()
    le6 = LabelEncoder()
    le7 = LabelEncoder()

    recommandations = []

    while not satisfait:
        # Interface utilisateur
        couleur = input("Quelle est votre couleur préférée parmi:" + rec["color"] + "? ")
        generation = int(input("Quelle est votre génération préférée parmi:" + rec["generation"] + "? "))
        type1 = input("Quel est votre type 1 préféré parmi:" + rec["type"] + "? ")
        type2 = input("Quel est votre type 2 préféré parmi:" + rec["type"].pop(type1) + "? ")
        legendary = bool(input("Aimez-vous les pokémons légendaires (True/False)? "))
        first_letter = input("Quelle est la première lettre de votre pokémon préféré? ")
        Nom=input("Quel est le nom de votre pokémon préféré parmi:" + rec["Name"] + "? ")


        # Créer un dataframe pour le nouveau Pokémon
        pokemon = pd.DataFrame([[Nom, type1, type2, generation, legendary, couleur, first_letter]], columns=["Name","Type 1", "Type 2", "Generation","legendary", "color","first_letter"])

        # Encoder les données
        pokemon["Name"] = le1.fit_transform(pokemon["Name"])
        pokemon["Type 1"] = le2.fit_transform(pokemon["Type 1"])
        pokemon["Type 2"] = le3.fit_transform(pokemon["Type 2"])
        pokemon["Generation"] = le4.fit_transform(pokemon["Generation"])
        pokemon["legendary"] = le5.fit_transform(pokemon["legendary"])
        pokemon["color"] = le6.fit_transform(pokemon["color"])
        pokemon["first_letter"] = le7.fit_transform(pokemon["first_letter"])


        # Prédire si le Pokémon sera favori ou non
        prediction1 = random_forest.predict(pokemon)
        prediction2 = neural_network.predict(pokemon)
        prediction3 = decision_tree.predict(pokemon)

        # Afficher la prédiction
        if prediction1[0] == 0 and prediction2[0] == 0 and prediction3[0] == 0:
            print("Le Pokémon sera favori")
            # Ajouter les pokémons recommandés à recommandations 
            for pokemon_id, pokemon_data in data.items():
                if (pokemon_data["couleur dominante"] == couleur 
                        and pokemon_data["tags"]["Generation"] == generation 
                        and ((pokemon_data["tags"]["Type 1"] == type1 and pokemon_data["tags"]["Type 2"] == type2) 
                        or (pokemon_data["tags"]["Type 1"] == type2 and pokemon_data["tags"]["Type 2"] == type1)) 
                        and not(pokemon_data["tags"]["Legendary"]^legendary)):
                    recommandations.append(pokemon_data)

            # Afficher les pokémons recommandés
            print("Voici les pokémons recommandés:")
            for id in recommandations:
                print(recommandations[id]["tags"]["Name"])

            satisfait = True
        else:
            print("Le Pokémon ne sera pas favori")
            satisfait = False

    # Afficher les pokémons recommandés
    print("Voici les pokémons recommandés:")
    for id in recommandations:
        print(recommandations[id]["tags"]["Name"])
        img = Image.open("./pokemon_jpg/" + id)


## Visualisation des données

In [ ]:
recommend_pokemon()

## Tests

In [ ]:
# Test des 3 modèles

data={"Name":"Pikachu","Type 1":"Electric","Type 2":"None","Generation":1,"legendary":False,"color":"Yellow","first_letter":"P"}

# Créer un dataframe pour le nouveau Pokémon
pokemon = pd.DataFrame([data], columns=["Name","Type 1", "Type 2", "Generation","legendary", "color","first_letter"])

# Encoder les données
pokemon["Name"] = le1.fit_transform(pokemon["Name"])
pokemon["Type 1"] = le2.fit_transform(pokemon["Type 1"])
pokemon["Type 2"] = le3.fit_transform(pokemon["Type 2"])
pokemon["Generation"] = le4.fit_transform(pokemon["Generation"])
pokemon["legendary"] = le5.fit_transform(pokemon["legendary"])
pokemon["color"] = le6.fit_transform(pokemon["color"])
pokemon["first_letter"] = le7.fit_transform(pokemon["first_letter"])

# Prédire si le Pokémon sera favori ou non
prediction1 = random_forest.predict(pokemon)
prediction2 = neural_network.predict(pokemon)
prediction3 = decision_tree.predict(pokemon)

# Afficher la prédiction
if prediction1[0]==0 :
    print("Le Pokémon sera favori")
else:
    print("Le Pokémon ne sera pas favori")
  
if prediction2[0] > 0.7 :
    print("Le Pokémon sera favori")
else:
    print("Le Pokémon ne sera pas favori")

if prediction3[0]==0 :
    print("Le Pokémon sera favori")
else:
    print("Le Pokémon ne sera pas favori")
    